© Copyright 2020 Anthony D. Dutoi

This file is part of TonyUtil.

TonyUtil is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.

## Concept / Theory

This module helps to implement a uniform conceptualiztion of a context-dependent evaluation of an expression or tree.  This could be thought of as a (massive) simplification of utilities found in the `ast` module of Python.  Its form is inspired by the concept of Lambda calculus.

In understanding its utility, it is important to start by pointing out how power and convenience are generally juxtaposed in the design of APIs.  The simpler a paradigm is, the easier it is to use, but less flexibility is the price usually payed for simplicity.  The idea here is to hit the sweet spot in making something that is powerful enough to do just what I need it to do, and not much more.

The idea is to be able to build nested objects (expressions or generic trees) whose evaluation then depends on some runtime context that is handed to it after it is completed.  The same tree could then be used to generate a variety of outputs from a single logical representation of a nested structure.  At an extreme end, this describes a programming language (in terms reminiscent of Lambda calculus), whereby a source code produces different results based on the context (input) which it is given.  There is clearly no reason to write a new programming language (which is why this is simpler than the `ast` module), but simple such a utility will still be valuable when a certain level of homoegeneity can be expected among the kinds of objects that are nested (for example, every node representing a mathematical operation, and the branching terminating at numbers).

## Implementation / Practice

The entire module consists of one class `tree_node`, meant to be used as a base class, and one function `evaluate`.

An instance of the `tree_node` class (more accurately, a child class thereof) maintains references to other objects, called subnodes.  These subnodes may be other instances of `tree_node` (building a branched tree), or other terminal objects that obey certain rules, discussed below.  Each subnode is identified by a label, which can be its dictionary key (or list index), if the subnodes are passed in at initialization time.  A subnode can also be added later with the concrete method `_append()`, meant for use by the implementation of the public API of the child class.  

    <instance>._append(subnode, label=None)

This adds a new `subnode`, associated with a given `label` (any hashable object).  If `label` is `None`, a sequential integer label is automatically generated.  Either way, the applied label is returned by this method.  Currently, using mixtures of specified and generated labels could result in an accidental name clash and raise an exception.  Presently, there is no plan to provide a way to remove subnodes or replace the subnode associated with a given label.  

The `tree_node` class expects one method to be implemented by a child class.

    <instance>._implementation(label, context)

This method takes the `context` and `label` (which define how the current object is to be evaluated) and uses it as a basis for constructing and independent `subcontext` (for downward communication to the subnodes that will be evaluated).  This is returned as the first component of a 2-tuple, `(subcontext, action)`.  The second component, `action`, is itself a function that takes a single argument.

    action(evaluated_subnodes)

This function uses the dictionary `evaluated_subnodes` (which associates the evaluated subnodes with their original labels) to evaluate the result for the current node, which shall be returned (for upward communication to the parent node).  Bear in mind that the current object's `label` and `context` are part of the enclosing scope of `action` and thus determine the way in which it is evaluated (which is the whole point).

Nodes (subnodes) are evaluated by the public function `evaluate()`.

    evaluate(node, label, context)

This simply calls the `_evaluate()` method of the argument `node`.

    <instance>._evaluate(label, context)

The argument `node` may be an instance of `tree_node` (for which this method is already defined), or in the case of bottoming out the recursion, any other class that has the method `_evaluate()` defined.  A tree built using instances of types derived from `tree_node` may therefore terminate with objects that are not `tree_node` instances, but which do have `_evaluate()` defined.  This should then return a concrete evaluation of the instance, in accordance with the values of `label` and `context`.  If `evaluate()` is called on a `node` argument that does not have a `_evaluate()` method defined for it, then `node` is passed as the first argument to a the `_evaluate_terminus()` method of the current `context`.

    <instance>._evaluate_terminus(node, label, context)

If this method will be relied upon (which can be convenient for allowing standard python data types to be termini of branches), then the implementor needs to ensure that the `_evaluate_terminus()` method is available in the `context` instance.

The values of `label` and `context` which are handed down recursively may be used or discarded at any level, according to need by the implementor.  `label` distinguishes the instance being evaluated by "name" from potential sibling subnodes of a common parent `tree_node` instance. `context` defines the context in which which the object will be evaluated, which is common to all siblings (the context is provided by the parent, perhaps after modification of its own context).  It is entirely up to the implementor to define the format of the contents of contexts (except that a `_evaluate_terminus()` method might be required).

In [1]:
from util.evaluation_tree import evaluate, tree_node

## Usage

Below is an absolutely trivial usage of the module, which does not take any advantage of the context dependence of the evaluation, but it is logically complete working code.

In [2]:
class dummy_node(tree_node):
    def __init__(self, subnodes):
        tree_node.__init__(self, subnodes)
    def _implementation(self, label, context):
        def evaluate_node(evaluated_subnodes):
            return "({})".format(", ".join(evaluated_subnodes.values()))
        return context, evaluate_node

class dummy_terminus(object):
    def __init__(self, value):
        self.value = value
    def _evaluate(self, label, context):
        return self.value

In [3]:
a = dummy_terminus("a")
b = dummy_terminus("b")
c = dummy_terminus("c")

A = dummy_node([a,b])
Z = dummy_node([A,c])

print(evaluate(Z))

((a, b), c)


Here is a minimal example, wherein the same tree generates two different outputs.

In [4]:
class capitalization_terminus(object):
    def __init__(self, value):
        self.value = value
    def _evaluate(self, label, context):
        if context=="A":
            return self.value.upper()
        else:
            return self.value

In [5]:
a = capitalization_terminus("a")
b = capitalization_terminus("b")
c = capitalization_terminus("c")

A = dummy_node([a,b])
Z = dummy_node([A,c])

print(evaluate(Z))
print(evaluate(Z, context="A"))

((a, b), c)
((A, B), C)


Now let's have some fun.  Let's create some simple expressions (using only + and -) which will return either the evaluated result or a printed string describing the expression, with proper parenthesization.  A better version of this (elsewhere) is one of the goals of this module.

In [6]:
from types import SimpleNamespace as struct

class arithmetic_expression(object):
    def __add__(self, other):
        return binary_op("+", self, other)
    def __sub__(self, other):
        return binary_op("-", self, other)

class binary_op(tree_node, arithmetic_expression):
    def __init__(self, op, arg1, arg2):
        tree_node.__init__(self, {1:arg1, 2:arg2})
        self.op = op
    def _implementation(self, label, context):
        def evaluate_node(evaluated_subnodes):
            parent_op = context.op
            arg1 = evaluated_subnodes[1]
            arg2 = evaluated_subnodes[2]
            return context.functions[self.op](arg1, arg2, parent_op, label)
        subcontext = struct(op=self.op, functions=context.functions, evaluate_terminus=context.evaluate_terminus)
        return subcontext, evaluate_node

class variable(arithmetic_expression):
    def __init__(self, number, formatter="{}"):
        self.number = number
        self.formatter = formatter

def evaluate_variable(var, label, context):
    parent_op = context.op
    return context.functions["x"](var.number, var.formatter, parent_op, label)

def print_plus(arg1, arg2, parent_op, arg_num):
    value = "{} + {}".format(arg1, arg2)
    if parent_op=="-" and arg_num==2:
        value = "({})".format(value)
    return value

def print_minus(arg1, arg2, parent_op, arg_num):
    value = "{} - {}".format(arg1, arg2)
    if parent_op=="-" and arg_num==2:
        value = "({})".format(value)
    return value

def print_var(number, formatter, parent_op, arg_num):
    value = formatter.format(number)
    if number<0 and arg_num==2:
        value = "({})".format(value)
    return value

def eval_plus(arg1, arg2, parent_op, arg_num):
    return arg1 + arg2

def eval_minus(arg1, arg2, parent_op, arg_num):
    return arg1 - arg2

def eval_var(number, formatter, parent_op, arg_num):
    return number

print_functions = {"x":print_var, "+":print_plus, "-":print_minus}
eval_functions  = {"x":eval_var,  "+":eval_plus,  "-":eval_minus}

print_context = struct(op="None", functions=print_functions, evaluate_terminus=evaluate_variable)
eval_context  = struct(op="None", functions=eval_functions, evaluate_terminus=evaluate_variable)

In [7]:
one    = variable(1)
Nthree = variable(-3)
five   = variable(5)
Nseven = variable(-7)
eleven = variable(11)

expr = Nseven + (five - eleven - (one + Nthree))

formula = evaluate(expr, context=print_context)
answer  = evaluate(expr, context=eval_context)
print("{}  =  {}".format(formula, answer))

-7 + 5 - 11 - (1 + (-3))  =  -11
